In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
import pandas as pd

# Increase precision of presented data for better side-by-side comparison
pd.set_option("display.precision", 8)

In [ ]:
print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__) #TensorFlow Hub is a repository of trained machine learning models ready for fine-tuning and deployable anywhere
print("Eager mode: ", tf.executing_eagerly()) #TensorFlow calculates the values of tensors as they occur in your code
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

from google.colab import drive
drive.mount('/content/drive')

data_root='/content/drive/MyDrive/COURSE PROJECT';

IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="validation",
shuffle=True,
target_size=IMAGE_SHAPE
)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="training",
shuffle=True,
target_size=IMAGE_SHAPE)

image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)


model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5",
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])

model.summary()

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator,
    epochs=20,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

saved_model_path = "./my_model.h5" # or you can simply use 'my_mode.h5'
model.save('/content/drive/MyDrive/LAmodelData') #save your model

saved_model_path = "saved_models/LearnerActions" # or you can simply use 'my_mode.h5'
model.save(saved_model_path) #save your model

from tensorflow import keras

# to reload your model
shoe_model = keras.models.load_model(saved_model_path)




Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Version:  2.14.0
Hub version:  0.15.0
Eager mode:  True
GPU is available
Mounted at /content/drive
/content/drive/MyDrive/COURSE PROJECT
Found 202 images belonging to 8 classes.
Found 828 images belonging to 8 classes.
Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 8)
['Gpt Reading' 'Gpt Generating' 'Gpt Writing' 'Jupyter Error'
 'Jupyter Running' 'Excel' 'Jupyter Success' 'Other']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              42626560  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 8)                 16392     
                                                                 
Total params: 42642952 (162.67 MB)
Trainable params

KeyboardInterrupt: ignored

In [ ]:
############################################################################
###############################################
###############################################
####### Get images and labels batch THAT NEEDS TO BE CATEGORIZED

import os
import cv2
import numpy as np
from tensorflow import keras


# Load the pre-trained model
shoe_model = keras.models.load_model(saved_model_path)

# Directory containing new images
new_images_directory = "/content/drive/MyDrive/v2_sample1"

# Load and preprocess images from the directory
image_files = os.listdir(new_images_directory)
images = [cv2.imread(os.path.join(new_images_directory, file)) for file in image_files]
images = [cv2.resize(img, (224, 224)) for img in images]  # Resize to match your model's input size
images = np.array(images) / 255.0  # Normalize pixel values

# Predict using the model
tf_model_predictions = shoe_model.predict(images)
print("Prediction results shape:", tf_model_predictions.shape)

# Create a directory to save individual images
output_directory = 'downloaded_images'
os.makedirs(output_directory, exist_ok=True)

# Save each image with its ID as the filename in high resolution
for i, prediction in enumerate(tf_model_predictions):
    image_filename = f"{output_directory}/image_{i + 1}_predicted_{np.argmax(prediction)}.png"
    plt.imshow(images[i])
    plt.axis('off')
    plt.savefig(image_filename, dpi=300, bbox_inches='tight')
    plt.clf()  # Clear the figure to avoid overlapping images

# Create a ZIP file containing the individual images
import shutil
shutil.make_archive(output_directory, 'zip', output_directory)

# Download the ZIP file
from google.colab import files
files.download(f'{output_directory}.zip')





# Load and preprocess images from the directory
image_files = os.listdir(new_images_directory)
images = [cv2.imread(os.path.join(new_images_directory, file)) for file in image_files]
images = [cv2.resize(img, (224, 224)) for img in images]  # Resize to match your model's input size
images = np.array(images) / 255.0  # Normalize pixel values

# Predict using the model
tf_model_predictions = shoe_model.predict(images)
print("Prediction results shape:", tf_model_predictions.shape)


tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels

print("Prediction results for the first elements")
tf_pred_dataframe.head()


predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]

In [ ]:


# Load and preprocess images from the directory
image_files = os.listdir(new_images_directory)
images = [cv2.imread(os.path.join(new_images_directory, file)) for file in image_files]
images = [cv2.resize(img, (224, 224)) for img in images]  # Resize to match your model's input size
images = np.array(images) / 255.0  # Normalize pixel values

# Predict using the model
tf_model_predictions = shoe_model.predict(images)
print("Prediction results shape:", tf_model_predictions.shape)

# Get predicted labels
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]

# ... (previous code remains unchanged)

# Extract only the numeric part from the file names
image_labels = [file.split('.')[0] for file in image_files]
image_labels = [''.join(filter(str.isdigit, label)) for label in image_labels]

# Create a DataFrame with Image Labels and Predicted Labels
results_df = pd.DataFrame({
    'Image_Label': image_labels,  # Extracting the numeric part from the filename
    'Predicted_Label': predicted_labels
})

# Convert the 'Image_Label' column to integers for sorting
results_df['Image_Label'] = results_df['Image_Label'].astype(int)

# Sort the DataFrame based on the 'Image_Label' column
results_df = results_df.sort_values(by='Image_Label')

# Save the DataFrame to a CSV file
results_df.to_csv('model_predictions_results.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('model_predictions_results.csv')



###from here onwards pat of old code
# Create a DataFrame with Image Labels and Predicted Labels
#results_df = pd.DataFrame({
#    'Image_Label': [file.split('.')[0] for file in image_files],  # Extracting the label from the filename
 #   'Predicted_Label': predicted_labels
#})

# Save the DataFrame to a CSV file
#results_df.to_csv('model_predictions_results.csv', index=False)

# Download the CSV file
#from google.colab import files
#files.download('model_predictions_results.csv')






In [ ]:
#

# Create a DataFrame with Image Labels, Predicted Labels, and Timestamps
results_df = pd.DataFrame({
    'Image_Label': image_labels,  # Using the sorted and renamed Image_Labels
    'Predicted_Label': predicted_labels
})

# Convert the 'Image_Label' column to integers for sorting
results_df['Image_Label'] = results_df['Image_Label'].astype(int)

# Sort the DataFrame based on the 'Image_Label' column
results_df = results_df.sort_values(by='Image_Label')

# Add Timestamps based on the identified learner actions
timestamps = []
current_action = None
start_time = None

for index, row in results_df.iterrows():
    if current_action is None:
        current_action = row['Predicted_Label']
        start_time = int(''.join(filter(str.isdigit, str(row['Image_Label']))))
    elif current_action != row['Predicted_Label']:
        end_time = int(''.join(filter(str.isdigit, str(row['Image_Label'])))) - 1
        timestamps.append({'Action': current_action, 'Start_Time': start_time, 'End_Time': end_time})
        current_action = row['Predicted_Label']
        start_time = int(''.join(filter(str.isdigit, str(row['Image_Label']))))

# Add the last action
end_time = int(''.join(filter(str.isdigit, str(row['Image_Label']))))
timestamps.append({'Action': current_action, 'Start_Time': start_time, 'End_Time': end_time})

timestamps_df = pd.DataFrame(timestamps)
timestamps_df.to_csv('learner_action_timestamps.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('learner_action_timestamps.csv')


In [ ]:
# Define a mapping of predicted labels to learner actions
def map_to_learner_actions(predicted_label):
    if predicted_label in ["Jupyter Success", "Jupyter Error", "Jupyter Running"]:
        return "Jupyter"
    elif predicted_label in ["Gpt Writing", "Gpt Reading", "Gpt Generating"]:
        return "Chat GPT"
    elif predicted_label == "excel":
        return "excel"
    elif predicted_label == "other":
        return "other"
    else:
        return "Unknown"

# Map the predicted labels to learner actions
learner_actions = [map_to_learner_actions(label) for label in predicted_labels]

# Extract and process image labels to use as time stamps
# Note: Assuming 'image_labels' are already defined and sorted in your existing code
time_stamps = [int(''.join(filter(str.isdigit, label))) for label in image_labels]

# Create a DataFrame with Time Stamps, Learner Actions, and Predicted Labels
results_df = pd.DataFrame({
    'Time Stamp': time_stamps,               # The renamed first column
    'Learner_Actions': learner_actions,      # The new second column with mapped actions
    'Predicted_Label': predicted_labels      # The third column
})

# No need to convert 'Time Stamp' column to integers since it's already done above
# Sort the DataFrame based on the 'Time Stamp' column, if needed
results_df = results_df.sort_values(by='Time Stamp')

# Save the DataFrame to a CSV file
results_df.to_csv('model_predictions_results.csv', index=False)

# Download the CSV file
#from google.colab import files
#files.download('model_predictions_results.csv')


In [ ]:
# Rename the 'Time Stamp' column to 'Image'
results_df.rename(columns={'Time Stamp': 'Image'}, inplace=True)

# Add another column for time stamps in seconds
results_df['Time (Seconds)'] = results_df['Image'] * 0.2  # Assuming each image represents 0.2 seconds

# Save the DataFrame to a CSV file
results_df.to_csv('model_predictions_results_updated.csv', index=False)

# Download the updated CSV file
from google.colab import files
files.download('model_predictions_results_updated.csv')